In [16]:
from itertools import product
from qiskit import QuantumCircuit
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

# ─── 1) SAT 문제 정의 ───────────────────────────────────
# 각 절은 정수 튜플로 표현합니다. 양수 i = x_{i-1}, 음수 -i = ¬x_{i-1}
clauses = [
    ( 1,  2,  3),   #  x0 ∨  x1 ∨  x2
    (-1,  2,  3),   # ¬x0 ∨  x1 ∨  x2
    ( 1, -2,  3),   #  x0 ∨ ¬x1 ∨  x2
    ( 1,  2, -3),   #  x0 ∨  x1 ∨ ¬x2
    (-1, -2, -3),   # ¬x0 ∨ ¬x1 ∨ ¬x2  ← 모두 1인 111만 제외
]

# 주어진 3비트 문자열 (bitstr: str) -> bool
def is_satisfied(bitstr: str) -> bool:
    for clause in clauses:
        if not any(
            (lit > 0 and bitstr[lit-1] == '1') or
            (lit < 0 and bitstr[abs(lit)-1] == '0')
            for lit in clause
        ):
            return False
    return True

# 고전적으로 가능한 해(비트스트링)를 모두 구함
all_sols = [b for b in map(''.join, product('01', repeat=3)) if is_satisfied(b)]
print("Classical solutions:", all_sols)


# ─── 2) Grover 오라클 생성: 주어진 고전 해 목록에 해당하는 상태만 위상반전
def apply_oracle(qc: QuantumCircuit, solutions: list[str]):
    n = qc.num_qubits
    for sol in solutions:
        # sol 비트스트링에서 '0'인 위치에 X를 걸어 모두 1로 만든 뒤
        for i, bit in enumerate(sol):
            if bit == '0':
                qc.x(i)
        # 마지막 큐비트를 표적(target)으로 다중제어 Z (phase-flip)
        qc.h(n-1)
        qc.mcx(list(range(n-1)), n-1)
        qc.h(n-1)
        # X를 다시 원위치로 되돌림
        for i, bit in enumerate(sol):
            if bit == '0':
                qc.x(i)


# ─── 3) Grover diffusion 연산: typical한 Grover 확산 연산
def apply_diffusion(qc: QuantumCircuit):
    n = qc.num_qubits
    qc.h(range(n))
    qc.x(range(n))
    qc.h(n-1)
    qc.mcx(list(range(n-1)), n-1)
    qc.h(n-1)
    qc.x(range(n))
    qc.h(range(n))


# ─── 4) 전체 Grover 회로 구성: 전체 Grover 회로 조립
def build_grover_for_sat(n: int, solutions: list[str], iterations: int = 1) -> QuantumCircuit:
    qc = QuantumCircuit(n)
    # 1) 균등 중첩 준비
    qc.h(range(n))
    # 2) Oracle + Diffusion 반복
    for _ in range(iterations):
        apply_oracle(qc, solutions)
        apply_diffusion(qc)
    qc.measure_all()
    return qc


# ─── 5) QPU 샘플링 함수 (Aachen)
def run_on_aachen(qc: QuantumCircuit, shots: int = 50) -> dict[str,int]:
    service = QiskitRuntimeService()
    backend = service.backend("ibm_aachen")
    sampler = Sampler(mode=backend)
    pm = generate_preset_pass_manager(optimization_level=3, backend=backend)
    isa = pm.run(qc)
    job = sampler.run([isa], shots=shots)
    res = job.result()[0]
    return res.data.meas.get_counts()


# ─── 6) Configuration Recovery (힐 클라이밍): 탈락한 비트에 대해서 한 자리 비트만 다른 neighbor 생성, 
# 이웃의 count > current count인 경우 current를 이웃으로 업데이트 하고 반복
def hill_climb_from(start: str, counts: dict[str,int]) -> str:
    current = start
    n = len(start)
    while True:
        improved = False
        for i in range(n):
            nbr = list(current)
            nbr[i] = '1' if nbr[i]=='0' else '0'
            nbr = ''.join(nbr)
            if counts.get(nbr, 0) > counts.get(current, 0):
                current = nbr
                improved = True
                break
        if not improved:
            return current
# ─── 7) counts에 값이 있는 모든 스트링을 내림차순으로 정렬하고, 각  b에 대해 hill_climb실행
# 안 중복인 해 결과 = recovered에 추가, 계속 추가하다가 len(recovered) == num_sols 되면 종료
def recover_multiple_solutions(counts: dict[str,int], num_sols: int = 3) -> list[str]:
    candidates = sorted(counts.keys(), key=lambda b: counts[b], reverse=True)
    recovered = []
    for b in candidates:
        sol = hill_climb_from(b, counts)
        if sol not in recovered:
            recovered.append(sol)
        if len(recovered) >= num_sols:
            break
    return recovered


# ─── Example Usage 
if __name__ == "__main__":
    n          = 3
    iterations = 1
    shots      = 100

    print("== Classical SAT solutions:", all_sols)

    qc     = build_grover_for_sat(n, all_sols, iterations)
    counts = run_on_aachen(qc, shots)
    print("Raw counts from QPU:", counts)

    # ─── 악의적 count test ──────────────────────────────────
    #evil_counts = {
     #   all_sols[0]: 80,   # 정답 #1
     #   all_sols[1]: 70,   # 정답 #2
      #  all_sols[2]: 5,    # 정답 #3 (의도적으로 적게)
       # '000':         20, # 비해 문자열에 노이즈
        #'010':         15,
       # '110':         10,
    #}

    # 초기 evil_counts 기반 top-3 후보
    #initial_candidates = sorted(
    #    evil_counts.keys(),
    #    key=lambda b: evil_counts[b],
    #    reverse=True
    #)[:3]
    #print("\n-- Initial top-3 candidates from evil_counts:", initial_candidates)

    # Configuration Recovery 적용
    #sols = recover_multiple_solutions(evil_counts, num_sols=3)
    sols = recover_multiple_solutions(counts, num_sols=3)
    print("Recovered top-3 candidates after recovery:", sols)




Classical solutions: ['011', '101', '110']
== Classical SAT solutions: ['011', '101', '110']
Raw counts from QPU: {'100': 6, '011': 32, '110': 22, '101': 25, '111': 11, '000': 3, '010': 1}
Recovered top-3 candidates after recovery: ['011', '101', '110']
